## Abdulazeez EDA fil 

## Statsbidrag per utbildningsområde (med och utan moms)

In [11]:
import pandas as pd

df_statsbidrag = pd.read_csv("data/statsbidrag_schablonnivåer.csv", encoding="utf-8")

df_statsbidrag.columns = df_statsbidrag.columns.str.strip()

print("Kolumner i CSV-filen:\n", df_statsbidrag.columns.tolist())


Kolumner i CSV-filen:
 ['Utbildningsområde', 'Utan momskompensation', 'Med momskompensation']


In [12]:
schablon_per_område = (
    df_statsbidrag.groupby("Utbildningsområde")[["Utan momskompensation", "Med momskompensation"]]
    .mean()
    .reset_index()
    .sort_values("Utbildningsområde")
)

schablon_per_område.index.name = "id"
schablon_per_område.reset_index(inplace=True)

print(schablon_per_område)


    id                             Utbildningsområde  Utan momskompensation  \
0    0                                       Data/IT                69900.0   
1    1       Ekonomi, administration och försäljning                62900.0   
2    2                      Friskvård och kroppsvård                74800.0   
3    3                 Hotell, restaurang och turism                64900.0   
4    4       Hälso- och sjukvård samt socialt arbete                67000.0   
5    5                  Journalistik och information                66600.0   
6    6                                       Juridik                60500.0   
7    7                      Kultur, media och design                84400.0   
8    8  Lantbruk, djurvård, trädgård, skog och fiske               110200.0   
9    9                    Pedagogik och undervisning                71300.0   
10  10                Samhällsbyggnad och byggteknik                70400.0   
11  11                             Säkerhetstjänster

In [13]:
import plotly.graph_objects as go

fig = go.Figure()
for col in ["Utan momskompensation", "Med momskompensation"]:
    fig.add_trace(go.Bar(
        x=schablon_per_område["Utbildningsområde"],
        y=schablon_per_område[col],
        name=col
    ))

fig.update_layout(
    title="Genomsnittligt schablonbelopp per utbildningsområde",
    barmode="group",
    xaxis_title="Utbildningsområde",
    yaxis_title="Schablonbelopp (kr)",
    xaxis_tickangle=-45
)
fig.show()


In [14]:
statsbidrag_total = pd.merge(
    beviljade,
    schablon_per_område,
    on="Utbildningsområde",
    how="left"
)

statsbidrag_total["Totalt statsbidrag utan moms"] = (
    statsbidrag_total["Totalt antal beviljade platser"] * statsbidrag_total["Utan momskompensation"]
)

statsbidrag_total["Totalt statsbidrag med moms"] = (
    statsbidrag_total["Totalt antal beviljade platser"] * statsbidrag_total["Med momskompensation"]
)

statsbidrag_total.head()


NameError: name 'beviljade' is not defined

In [ ]:
import plotly.express as px

fig = px.bar(
    statsbidrag_total,
    x="Utbildningsområde",
    y="Totalt statsbidrag med moms",
    title="Totalt statsbidrag per utbildningsområde (med moms)",
    labels={"Totalt statsbidrag med moms": "Statsbidrag (kr)"},
    color="Utbildningsområde"
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [16]:
total_utan_moms = df_statsbidrag["Totalt statsbidrag utan moms"].sum()
total_med_moms = df_statsbidrag["Totalt statsbidrag med moms"].sum()

print(f"Totalt statsbidrag utan moms: {total_utan_moms:,.0f} kr")
print(f"Totalt statsbidrag med moms: {total_med_moms:,.0f} kr")


Totalt statsbidrag utan moms: 1,032,200 kr
Totalt statsbidrag med moms: 1,093,500 kr


In [22]:
kpi_full = pd.merge(
    kpi,
    ansökta_platser_utbildningsområde[["Sökt utbildningsområde", "Total ansökta platser"]],
    left_on="Utbildningsområde",
    right_on="Sökt utbildningsområde",
    how="left"
)


kpi_full["Beviljandegrad (%)"] = (
    kpi_full["Beviljade platser"] / kpi_full["Total ansökta platser"] * 100
).round(1)


kpi_full.drop(columns=["Sökt utbildningsområde"], inplace=True)


kpi_full = kpi_full.sort_values("Totalt statsbidrag (med moms)", ascending=False)


kpi_full.reset_index(drop=True, inplace=True)
kpi_full



NameError: name 'kpi' is not defined